# **Mounts your drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Prepare data
===

In [0]:
!pip install tifffile

In [0]:
# Functions
import numpy
import sys
sys.path.append("/content/drive/My Drive/.../unet_segmentation/py_files") # path to py_files folder
from helpers import *
from data_loading import *

# test diff

# Autoreload
%load_ext autoreload
%autoreload 2

# Set random seed
np.random.seed(1)

# Load raw data from the Cell Tracking Challenge http://celltrackingchallenge.net/2d-datasets/. Download first the data.
data, labels = load_data("/content/drive/My Drive/.../unet_segmentation/data/raw/hela/image/*.tif",
                         "/content/drive/My Drive/.../unet_segmentation/data/raw/hela/label/*.tif") # Set correct paths
for i in range(len(labels)):
  tmp = np.array(labels[i])
  tmp[tmp > 0] = 255
  tmp[tmp == 0] = 0
  tmp = tmp.astype('uint8')
  tmp = Image.fromarray(tmp, 'L')
  labels[i] = tmp
  
# Split the data into train and test
X_train, y_train, X_test, y_test = split_data(data, labels, ratio = 0.5)

# Set the paths and create the folders to save preprocessed data as .png
TRAIN_DIR="/content/drive/My Drive/unser_project/data/processed/hela/train/"
TEST_DIR="/content/drive/My Drive/unser_project/data/processed/hela/test/"

if not os.path.exists(TRAIN_DIR):
    os.makedirs(TRAIN_DIR+"/image/")
    os.makedirs(TRAIN_DIR+"/label/")
    
if not os.path.exists(TEST_DIR):
    os.makedirs(TEST_DIR+"/image/")
    os.makedirs(TEST_DIR+"/label/")
    
# Save train and test files
save_data(X_train, y_train, TRAIN_DIR)
save_data(X_test, y_test, TEST_DIR)

# **Imports modules**

In [0]:
import sys
sys.path.append("/content/drive/My Drive/.../unet_segmentation/py_files") # path to py_files folder
!pip install tifffile
!pip install --upgrade tensorflow
!pip install --upgrade keras
from model import *
from convert_to_pb import *
from data_loading import *
from helpers import *
from unet_weights import *
from fit_model import *
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image, ImageOps, ImageFilter
import pickle
from test import *
import cv2 as cv

# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Set random seed
np.random.seed(1)

Train U-Net
===

## Hela cells

In [0]:
# Load training and validation data
# Note that the subset of generator used for the training generator is "validation" because we don't want to augment our data
# Specify paths where inside there are "image" and "label" folder
trainGen = dataGenerator(batch_size = 2, subset = "train", path = '/content/drive/My Drive/.../unet_segmentation/data/processed/hela/train')
validGen = dataGenerator(batch_size = 1, subset = "validation", path = '/content/drive/My Drive/.../unet_segmentation/data/processed/hela/test')

In [0]:
model = unet()

# Callbacks
model_checkpoint = ModelCheckpoint('/content/drive/My Drive/.../unet_segmentation/models/{b}.hdf5'.format(b="unet_hela"), monitor='val_loss', verbose=1, save_best_only=True)

# Fit
history = model.fit_generator(trainGen,
                          steps_per_epoch=500,
                          epochs=1,
                          callbacks=[model_checkpoint], 
                          validation_data = validGen, 
                              validation_steps = 9)

Results
===

## Hela cells

In [0]:
# Define paths.
    path_to_model = '/content/drive/My Drive/.../unet_segmentation/models/unet_hela.hdf5'
   
    # Load model.
    model = load_model(path_to_model)
    # Load training and validation data
    # Note that the subset of generator used for the training generator is "validation" because we don't want to augment our data
    print("Validation")
    validGen = dataGenerator(batch_size = 1, subset = "validation", path = '/content/drive/My Drive/.../unet_segmentation/data/processed/hela/test')
    
    accuracies = model.evaluate_generator(validGen, steps=9, verbose=1) 
    print(accuracies)
    
    print("Training")
    trainGen = dataGenerator(batch_size = 1, subset = "validation", path = '/content/drive/My Drive/.../unet_segmentation/data/processed/hela/train')
    
    accuracies = model.evaluate_generator(validGen, steps=8, verbose=1) 
    print(accuracies)
    

Prediction
===

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.contrib.saved_model import save_keras_model
import tensorflow.keras
from keras.models import load_model
testGen = dataGenerator(batch_size = 1, subset = "test", path = '/content/drive/My Drive/.../unet_segmentation/data/processed/hela/test')
model = unet()
model = load_model('/content/drive/My Drive/.../unet_segmentation/models/unet_hela.hdf5')
results = model.predict_generator(testGen,9,verbose=1, workers=1)
#saveResults('/content/drive/My Drive/.../unet_segmentation/data/hela/test/', results, convert = True)
#saveResults('/content/drive/My Drive/.../unet_segmentation/data/hela/test/', results, convert = False)

In [0]:
from sklearn.metrics import jaccard_similarity_score
  
acc_tot = [];

for i in range(9):
  label = cv.imread('/content/drive/My Drive/.../unet_segmentation/data/hela/test/label/0{b}.png'.format(b=i))
  label = cv.resize(label, (256,256))
  acc = jaccard_similarity_score(label[...,0].flatten(), convertLabel(results[i]).flatten())
  acc_tot.append(acc)
     
print("Jaccard average : {b}".format(b=np.mean(acc_tot)))